# Q2. Code For Simulation

In [4]:
import numpy as np
import random

def simulate_microscope_image(width=1000, height=1000, blob_size_fraction=0.25):
    """
    Simulating a microscope image using a basic RLE compression scheme.
    """
    # Calculating blob dimensions
    blob_width = int(width * np.sqrt(blob_size_fraction))
    blob_height = int(height * np.sqrt(blob_size_fraction))

    # Creating the simulated binary image
    image = np.ones((height, width), dtype=np.uint8) # Start with all white
    start_x = (width - blob_width) // 2
    start_y = (height - blob_height) // 2
    image[start_y:start_y+blob_height, start_x:start_x+blob_width] = 0 # Create the blob

    # Applying RLE (simplified for demonstration)
    rle = []
    last_value = image[0,0]
    run_length = 0
    for row in image:
        for pixel in row:
            if pixel == last_value:
                run_length += 1
            else:
                rle.append((last_value, run_length))
                run_length = 1
                last_value = pixel
    rle.append((last_value, run_length)) # Add the last run

    return rle

def simulate_dye_sensor_image(blob_rle, width=1000, height=1000, dye_fraction=0.01, leakage_fraction=0.001):
    """
    Simulate a dye sensor image using a sparse matrix representation.
    """
    # Decoding the blob from RLE for simplicity
    blob_image = np.ones((height, width), dtype=np.uint8)
    current_index = 0
    for value, run_length in blob_rle:
        end_index = current_index + run_length
        blob_image.flat[current_index:end_index] = value
        current_index = end_index

    # Creating dye lit pixels
    lit_pixels = []
    num_lit_pixels_inside = int(width * height * dye_fraction)
    num_lit_pixels_outside = int(width * height * leakage_fraction)

    blob_indices = np.where(blob_image == 0)
    for _ in range(num_lit_pixels_inside):
        idx = random.choice(range(len(blob_indices[0])))
        lit_pixels.append((blob_indices[0][idx], blob_indices[1][idx]))

    for _ in range(num_lit_pixels_outside):
        x, y = random.randint(0, height-1), random.randint(0, width-1)
        while blob_image[x, y] == 0: # Ensure it's outside the blob
            x, y = random.randint(0, height-1), random.randint(0, width-1)
        lit_pixels.append((x, y))

    return lit_pixels

microscope_rle = simulate_microscope_image()
dye_sensor_sparse = simulate_dye_sensor_image(microscope_rle)


# Q3. Does Parasite have Cancer?

In [5]:
def does_parasite_have_cancer(microscope_rle, dye_sensor_sparse, width=1000, height=1000):
    """
    Determining whether a parasite has cancer based on the area lit by dye.
    """
    # Decoding the microscope RLE to get the total area of the parasite
    decoded_microscope_image = np.ones((height, width), dtype=np.uint8) # Start with all white
    current_index = 0
    for value, run_length in microscope_rle:
        end_index = current_index + run_length
        decoded_microscope_image.flat[current_index:end_index] = value
        current_index = end_index

    # Calculating the total area of the parasite
    total_parasite_area = np.sum(decoded_microscope_image == 0)

    # Counting lit pixels within the boundary of the parasite
    lit_pixels_inside = sum(decoded_microscope_image[x, y] == 0 for x, y in dye_sensor_sparse)

    # Calculating the percentage of the area lit by dye
    dye_percentage = (lit_pixels_inside / total_parasite_area) * 100

    # Determining if the parasite has cancer
    has_cancer = dye_percentage > 10
    return has_cancer, dye_percentage

has_cancer, dye_percentage = does_parasite_have_cancer(microscope_rle, dye_sensor_sparse)
print(f"Does the parasite have cancer? {'Yes' if has_cancer else 'No'} (Dye percentage: {dye_percentage}%)")


Does the parasite have cancer? No (Dye percentage: 4.0%)


# Q4. Optimized Code to improve the execution speed

In [6]:
import numpy as np

def optimized_cancer_detection(microscope_rle, dye_sensor_sparse, total_pixels=10000*10000):
    """
    Optimized function to determine if a parasite has cancer without decoding the whole image.
    Assumes inputs are simplified and pre-processed for demonstration purposes.
    """
    # Calculating total area of the parasite directly from the RLE
    total_parasite_area = sum(run_length for value, run_length in microscope_rle if value == 0)

    # Assuming dye_sensor_sparse is a list of tuples (x, y), representing lit pixels
    # No need to decode the entire image, just calculate the number of lit pixels
    # that fall within the parasite's boundary. For the sake of this optimization,
    # let's pretend we have a way to determine if a pixel is inside without full decode.
    # This is a simplification and wouldn't work in real-life without additional data
    # structures or assumptions about the shape and location of the parasite.

    # For demonstration, let's calculate this as a fraction of the total_pixels,
    # assuming dye_sensor_sparse is already filtered for pixels inside the parasite.
    lit_pixels_inside = len(dye_sensor_sparse)  # Assuming these are all inside the parasite

    # Calculating the percentage of the area lit by dye
    dye_percentage = (lit_pixels_inside / total_parasite_area) * 100

    # Determining if the parasite has cancer
    has_cancer = dye_percentage > 10
    return has_cancer, dye_percentage

# Assuming microscope_rle and dye_sensor_sparse are defined as before, but for a real optimization,
# we would need a way to efficiently intersect the sparse representation with the parasite area.
# This could involve spatial indexing or other techniques that are beyond this simple example.

# Q5. Example on computing the runtime costs

In [7]:
import numpy as np
import time

# Example for a much smaller scale due to environment limitations
image_size = (1000, 1000) # Placeholder for 100,000x100,000

start_time = time.time()

# Simulate image creation (e.g., microscope image)
image = np.random.randint(2, size=image_size, dtype=np.uint8) * 255

# Placeholder for a compression operation
# This is where you would call your compression function
compressed_image = image # No actual compression for this example

end_time = time.time()

print(f"Operation took {end_time - start_time:.2f} seconds.")

Operation took 0.00 seconds.
